# **데이터 수집 및 탐색**

## **데이터 수집**

### **패키지 라이브러리 불러오기**

* 패키지를 자동으로 설치한 뒤 다음의 알고리즘 실행에 필요한 여러가지 패키지와 라이브러리를 불러옵니다.

In [ ]:
# 패키지 설치

!pip install astropy
!pip install regions
!pip install sklearn
!pip install pandas
!pip install photutils
!pip install jupyter_contrib_nbextensions && jupyter contrib nbextension install --user # 쥬피터 노트북 각종 편의 옵션 제공

In [ ]:
# 패키지 라이브러리 불러오기

%matplotlib notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

from astropy.wcs import WCS
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
from astropy.convolution import Gaussian2DKernel, convolve, convolve_fft

from regions import PixCoord, CirclePixelRegion

from photutils.aperture import EllipticalAperture
from photutils.isophote import EllipseGeometry, Ellipse

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import click2label
import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
import numpy as np

### **분석할 데이터 불러오기**

* 운영체제 윈도우(Windows) 기준, C 드라이브에 ring_galaxy 라는 이름의 폴더를 만든 후, Github에서 다운로드한 모든 외부고리 은하 fits 파일과 ring_galaxy.csv 파일을 이 폴더에 저장해놓습니다.
* 또한 ring_galaxy 라는 폴더 안에 result 라는 이름의 폴더를 만들어 놓습니다. 이 폴더에는 데이터 처리 및 분석시 얻은 결과물이 저장됩니다.

In [ ]:
# 불러올 데이터의 위치 경로 및 분석 결과 일괄 저장 경로 지정

often_path = 'C:\\ring_galaxy\\' #  고리은하 fits 파일과 csv 파일 불러오는 경로 지정
save_path = 'C:\\ring_galaxy\\result\\' # 분석 결과 일괄 저장 경로 지정

* 고리은하 목록을 불러옵니다.

In [ ]:
# 고리은하 csv 파일로부터 고리은하 목록 불러와 확인하기

pd.set_option('display.max_rows', None)
ring_df = pd.read_csv(often_path + 'ring_galaxy.csv') # 고리은하 csv 파일 불러오기
display(ring_df)

* 분석하려는 고리은하 번호(object number)를 입력합니다.(예: 1010). 그러면 해당 은하의 적경(ra), 적위(dec) 좌표와 fits 파일이 불러와 집니다.

In [ ]:
# 분석하려는 고리은하 번호 입력

ring_num = int(input('분석하려는 고리은하 번호(object_number)를 입력하세요.')) # 분석할 고리은하 번호 입력
ring = ring_df['object_number'] == ring_num
ring_info = ring_df[ring]
display(ring_info) # 선택한 고리은하의 데이터 프레임 정보 출력
ring_ra = ring_info['ra'].values[0] # 선택한 고리은하 적경 값
ring_dec = ring_info['dec'].values[0] # 선택한 고리은하 적위 값

In [ ]:
# 각 필터 fits 파일 불러오기

u = fits.open(often_path + str(ring_num)+'_u.fits') # u필터 fits 파일 불러오기
g = fits.open(often_path + str(ring_num)+'_g.fits') # g필터 fits 파일 불러오기
r = fits.open(often_path + str(ring_num)+'_r.fits') # r필터 fits 파일 불러오기
i = fits.open(often_path + str(ring_num)+'_i.fits') # i필터 fits 파일 불러오기
z = fits.open(often_path + str(ring_num)+'_z.fits') # z필터 fits 파일 불러오기

filter = ['u', 'g', 'r', 'i', 'z'] # 필터 모음

## **데이터 탐색(EDA)**

### **데이터 기본 통계 정보**

* 분석하려는 은하의 fits 파일의 헤더 정보와 데이터 구조, 기본 통계 정보를 확인합니다.

In [ ]:
# 각 필터 fits 파일 정보

for f in filter:
    print(globals()['{}'.format(f) ].info())
    print('------------------------------------------------')

In [ ]:
# 각 필터 fits 파일 헤더, 데이터 상세 정보

for f in filter:
    globals()['hdr_{}'.format(f) ] = globals()[ '{}'.format(f) ][0].header # 헤더 정보
    globals()['dt_{}'.format(f) ] = globals()[ '{}'.format(f) ][0].data # 데이터 정보

print(hdr_u)
print('--------------------------------------------------')
print(dt_u) 

In [ ]:
# 각 필터 fits 파일 데이터 통계 정보

for f in filter:
    print('데이터 구조_'+f+':', globals()[ 'dt_{}'.format(f) ].shape) # 행렬
    print('데이터 최소값_'+f+':', np.min( globals()[ 'dt_{}'.format(f)])) # 최소값
    print('데이터 최대값_'+f+':', np.max( globals()[ 'dt_{}'.format(f)])) # 최대값
    print('데이터 평균_'+f+':', np.mean( globals()[ 'dt_{}'.format(f)])) # 평균값
    print('데이터 중앙값_'+f+':', np.median( globals()[ 'dt_{}'.format(f)])) # 중앙값
    print('데이터 표준편차값_'+f+':', np.std( globals()[ 'dt_{}'.format(f) ])) # 표준준편차값
    print('--------------------------------------------')

### **이미지 시각화**

* 분석하려는 은하의 fits 파일에 담긴 적경, 적위 위치 좌표를 컴퓨터가 이미지로써 인식이 가능하도록 픽셀 좌표로 변환합니다. 
* fits 파일의 원본 전체 이미지 속에서 분석하려는 은하의 위치를 확인합니다. 

In [ ]:
# 각 필터 fits 파일 천체 이미지의 중심 픽셀 및 천문 좌표 정보

for f in filter:
    globals()[ 'wcs_{}'.format(f) ] = WCS(globals()[ 'hdr_{}'.format(f) ])
    print( 'wcs_'+f+':', globals()[ 'wcs_{}'.format(f) ] )
    print( '-----------------------------------------' )

In [ ]:
# 각 필터 fits 파일 관찰 대상의 천문 좌표를 픽셀 좌표로 변경한 정보

if ring_num == 1401: # 1401 고리은하의 천문 좌표 -> 픽셀 좌표 변환
    for f in filter:
        ob_wcs = [213.3, 8.8]
        globals()['ob_pix_{}'.format(f)] = [350,1037]
        print('ob_pix_'+f+':', globals()['ob_pix_{}'.format(f) ][0], globals()[ 'ob_pix_{}'.format(f) ][1])
elif ring_num == 1450: # 1450 고리은하의 천문 좌표 -> 픽셀 좌표 변환
    for f in filter:
        ob_wcs = [228.3, 5.7]
        globals()['ob_pix_{}'.format(f)] = [729, 1124]
        print('ob_pix_'+f+':', globals()['ob_pix_{}'.format(f) ][0], globals()[ 'ob_pix_{}'.format(f) ][1])
else: # 그 외 모든 고리은하 천문 좌표 -> 픽셀 좌표 변환   
    ob_wcs1 = ring_ra 
    ob_wcs2 = ring_dec
    print('적경(ra):', ob_wcs1)
    print('적위(dec):', ob_wcs2)
    ob_wcs = [ob_wcs1, ob_wcs2]
    for f in filter:
        globals()['ob_pix_{}'.format(f)] = np.around(globals()['wcs_{}'.format(f)].world_to_pixel_values(ob_wcs[0], ob_wcs[1] )).astype('int32')
        print('픽셀 좌표_'+f+':', globals()[ 'ob_pix_{}'.format(f)][0], globals()['ob_pix_{}'.format(f)][1])

In [ ]:
# 전체 이미지 시각화

for f in filter: # 이미지 밝기 백분율
    globals()['max_value_{}'.format(f)] = np.percentile(globals()[ 'dt_{}'.format(f) ], 99.8) # 최대갓 백분율
    globals()['min_value_{}'.format(f)] = np.percentile(globals()[ 'dt_{}'.format(f) ], 15) # 최솟값 백분율

fig = plt.figure(figsize = (20,18)) # 전체 이미지 크기
n = 1
for f in filter:
    ax = fig.add_subplot(4,3,n, projection = wcs_u) # 천문좌표 중심 이미지 시각화
    im1 = plt.imshow(globals()['dt_{}'.format(f)], cmap = 'gray_r', vmax = globals()['max_value_{}'.format(f)], vmin = globals()['min_value_{}'.format(f)], origin = 'lower') # 전체 이미지
    ax.scatter(ob_wcs[0], ob_wcs[1], transform = ax.get_transform('fk5'), s = 300, edgecolor = 'red', facecolor = 'none', linewidth = 2) # 고리은하 위치 표시
    plt.grid(color = 'white', ls = '--') 
    plt.colorbar(im1) 
    plt.title(f +'_ra '+str(ob_wcs[0])+', dec '+str(ob_wcs[1])) 
    
    ax = fig.add_subplot(4,3,n+6) # 픽셀 좌표 중심 이미지 시각화
    im2 = plt.imshow( globals()['dt_{}'.format(f)], cmap = 'gray_r', vmax = globals()['max_value_{}'.format(f)], vmin = globals()[ 'min_value_{}'.format(f)], origin = 'lower') # 전체 이미지
    plt.plot( globals()['ob_pix_{}'.format(f)][0], globals()[ 'ob_pix_{}'.format(f)][1], 'o', ms = 20, mec='red', mfc='none', linewidth=5) # 고리은하 위치 표시
    plt.grid(color = 'white', ls = '--') 
    plt.colorbar(im2) # 컬러바
    plt.title(f +'_pix '+str( globals()['ob_pix_{}'.format(f)][0])+', '+str( globals()['ob_pix_{}'.format(f)][1])) 
    n+=1
plt.show()

# **데이터 처리**

## **데이터 변환**

### **이미지 변환**

* 원본 전체 이미지 중 분석하려는 은하만 포착하고 시각화 합니다.
* 이때 분석하기에 적합하도록 은하 이미지를 300 x 300 픽셀 이하의 정사각형 구조로 적절히 입력합니다. 기본값을 통한 자동 조절을 원하시면 엔터를 누르십시오.

In [ ]:
# 원본 전체 이미지 중 분석 대상 은하만 포착 및 시각화

vmax = 90. 
vmin = 15. 
print('max_percetile:', vmax)
print('min_percetile:', vmin)

fig = plt.figure(figsize = (14,9))
n = 1
for f in filter:
    globals()['max_value_{}_temp'.format(f)] = np.percentile(globals()['dt_{}'.format(f) ], vmax)
    globals()['min_value_{}_temp'.format(f)] = np.percentile(globals()['dt_{}'.format(f) ], vmin)
    ax = fig.add_subplot( 2,3,n )
    plt.imshow(globals()['dt_{}'.format(f)], cmap = 'gray_r', vmax = globals()['max_value_{}'.format(f)], vmin = globals()['min_value_{}'.format(f)], origin = 'lower')
    plt.grid(color = 'white', ls = '--')
    plt.colorbar()
    plt.xlim(globals()['ob_pix_{}'.format(f) ][0]-150, globals()['ob_pix_{}'.format(f) ][0]+150) # 고리은하 중심 x축 300픽셀
    plt.ylim(globals()['ob_pix_{}'.format(f) ][1]-150, globals()['ob_pix_{}'.format(f) ][1]+150) # 고리은하 중심 y축 300픽셀
    plt.title(f+'_pix '+str(globals()['ob_pix_{}'.format(f)][0])+', '+str(globals()['ob_pix_{}'.format(f)][1]))
    n+=1
plt.show()

In [ ]:
# 원본 축소 조절 값 일력 및 결과 통계 수치로 확인 & 시각화

try: # 분석하고자 하는 픽셀구조 입력 (기본값은 300 픽셀 * 300 픽셀)
    shape_x, shape_y = map(int, input('쉼표(,)를 이용해 분석에 적합하도록 정사각형의 픽셀 구조를 입력하세요. 기본값을 통한 자동 조절을 원하시면 엔터를 누르십시오.(기본값: 300, 300): ').split(',')) 
    print('입력한 이미지 픽셀 구조: ', shape_x, shape_y)
except:
    shape_x = 300
    shape_y = 300
    print('기본값에 의한 이미지 픽셀 구조:', shape_x, shape_y)

print('------- 분석하려는 은하가 모퉁이에 있는 경우 컴퓨터의 자동계산에 의해 입력한 픽셀 구조 보다 작게 될 수 있습니다. --------')

all_shape = []    
for f in filter:
    for i in reversed(range(50, shape_x+10, 10)): # 은하가 모퉁이에 있으면 이미지가 짤리기 때문에, 이미지가 정사각형이 되게끔  x축과 y축이 같아 질때 까지 짝수 단위로 축소 
        globals()['ob_dt_{}'.format(f)] = globals()['dt_{}'.format(f)][globals()['ob_pix_{}'.format(f)][1] - int(i/2) : globals()['ob_pix_{}'.format(f) ][1] + int(i/2), globals()['ob_pix_{}'.format(f)][0] - int(i/2) : globals()['ob_pix_{}'.format(f)][0] + int(i/2)]
        if globals()['ob_dt_{}'.format(f)].shape[0] == globals()['ob_dt_{}'.format(f)].shape[1]:
            all_shape.append(globals()['ob_dt_{}'.format(f)].shape)
            break
            
shape1 = np.min(all_shape) 
shape2 = np.min(all_shape)

center = [int(shape1/2), int(shape2/2)] # 이미지의 중심 좌표 출력(이미지가 정사각형이니 x축 중심, y축 중심 같음)
print('초기 이미지 중심 좌표:', center)
    
for f in filter: # 탐색된 정사각형 행렬로 이미지 조정
    globals()['ob_dt_{}'.format(f)] = globals()['dt_{}'.format(f)][globals()['ob_pix_{}'.format(f)][1] - int(shape1/2) : globals()['ob_pix_{}'.format(f)][1] + int(shape1/2), globals()['ob_pix_{}'.format(f)][0] - int(shape2/2) : globals()['ob_pix_{}'.format(f)][0] + int(shape2/2)] 
    print('이미지 픽셀 구조_'+f+':', globals()[ 'ob_dt_{}'.format(f) ].shape) 
        
# 그림
vmax = 90.
vmin = 15.
print('max_percetile:', vmax)
print('min_percetile:', vmin)

fig = plt.figure(figsize = (14,9))
n = 1
for f in filter:
    globals()['max_value_{}'.format(f)] = np.percentile(globals()['ob_dt_{}'.format(f)], vmax)
    globals()['min_value_{}'.format(f)] = np.percentile(globals()['ob_dt_{}'.format(f)], vmin)
    ax = fig.add_subplot( 2,3,n )
    plt.imshow(globals()['ob_dt_{}'.format(f)], cmap = 'gray_r', vmax = globals()['max_value_{}'.format(f)], vmin = globals()['min_value_{}'.format(f)], origin = 'lower')
    plt.plot(center[1], center[0], '+', ms = 10, mec='red', mfc='red', linewidth=2)
    plt.grid(color = 'white', ls = '--')
    plt.colorbar()
    plt.title(f+'_pix '+str(center[0])+', '+str(center[1]))
    n+=1
plt.show()

## **배경 하늘의 밝기 제거(skysub)**

### **배경 하늘 평균 값 추정**

* 배경 하늘 값 추정은 두 가지 방식으로 이루어집니다.
* 첫 번째는 이미지를 36개의 구역으로 균등하게 나눠, 평균 값이 가장 적은 5개 구역을 선별하고 그것을 평균화하여 배경 하늘 값을 추정합니다. 
  이를 '5개 최소구역평균 배경하늘 값' 이라고 명명합니다.
* 두 번째는 이미지에서 신호 대 잡음비(Signal Noise Ratio, SNR)가 1.017배가 넘는 천체는 마스킹을 한 뒤 그 나머지를 평균화 하여 배경 하늘 값을 추정합니다.
  이를 '천체마스킹평균 배경하늘 값'이라고 명명합니다.  

In [ ]:
# 각 필터의 이미지를 36개의 구역으로 균등하게 나눠, 평균 값이 가장 적은 5개 구역을 선별, 5개 구역의 평균으로 배경 하늘 값 추정

f_1 = 'i'
for f in filter:
    globals()['partition_{}'.format(f)] = []
    globals()['noise_mean_{}'.format(f)] = []

n = 1
for f in filter:
    for j in range(0,6):
        for k in range(0,6):
            globals()['partition_noise_{}'.format(f)] = globals()['ob_dt_{}'.format(f)][int(shape1/6)*j : int(shape1/6)*(j+1), int(shape2/6)*k : int(shape2/6)*(k+1)]
            globals()['partition_noisemean_{}'.format(f)] = np.mean(globals()['partition_noise_{}'.format(f)])
            globals()['partition_{}'.format(f)].append(globals()['partition_noise_{}'.format(f)])
            globals()['noise_mean_{}'.format(f)].append(globals()['partition_noisemean_{}'.format(f)])
            n += 1

for f in filter:
    globals()['nosie_mean_least5_{}'.format(f)] = np.sort(globals()['noise_mean_{}'.format(f)][0:5])
    globals()['sky_nosie_mean_least5_mean_{}'.format(f)] = np.mean(globals()['nosie_mean_least5_{}'.format(f)])
    print('5개 구역의 평균 하늘 값_' + f + ':' , globals()['nosie_mean_least5_{}'.format(f)])
    print('5개 구역의 평균 하늘 값을 평균화 한 값_' + f + ':', globals()['sky_nosie_mean_least5_mean_{}'.format(f)])
    print('------------------------------------------')

In [ ]:
# I 필터 기준 이미지에서 snr이 1.017배가 넘는 천체는 마스킹을 한 뒤 그 나머지를 모두 배경 하늘 값으로 추정

# 배경 하늘 값 추출시 제외 영역
snr = 1.017
globals()['exc_ob_{}'.format(f_1)] = np.where( globals() [ 'ob_dt_{}'.format(f_1) ] >= globals()['sky_nosie_mean_least5_mean_{}'.format(f_1)] * snr )
print(str(f_1)+' filter 잡음 영역 평균'+'('+str(globals()['sky_nosie_mean_least5_mean_{}'.format(f_1)])+')'+'*SNR'+'('+str(snr)+'):', globals()['sky_nosie_mean_least5_mean_{}'.format(f_1)] * snr )
print( '-----------------------------------------' )

exc_ob_cor= globals()['exc_ob_{}'.format(f_1)]
exc_ob_rad= 10
print('제외한 천체(객체)들의 반경(픽셀):', exc_ob_rad)
print( '-----------------------------------------' ) 
        
# 배경 하늘 값 추출시 제외 영역 마스크 지정
exc_mask_reg = []
exc_aperture_reg = []
n = 1
for j,k in zip(exc_ob_cor[1], exc_ob_cor[0]):
    exc_center = PixCoord(j, k)
    #print('%d_[x,y,r]:'%n, j, k, exc_ob_rad )
    exc_aperture = CirclePixelRegion(exc_center, exc_ob_rad)
    exc_mask = exc_aperture.to_mask(mode = 'center')
    exc_aperture_reg.append(exc_aperture)
    exc_mask_reg.append(exc_mask.bbox)
    n += 1

# 데이터 사본(copy) 생성
for f in filter:
    globals()['ob_dt_{}_c'.format(f)] = globals()['ob_dt_{}'.format(f)].copy()

# 배경 하늘 값 추출시 제외한 영역 0 변환(마스킹) 및 배경 하늘 값 평균
for f in filter:
    globals()['ob_dt_{}_c'.format(f)] = globals()['ob_dt_{}'.format(f)].copy()
    for i in range(len(exc_mask_reg)): 
        globals()['ob_dt_{}_c'.format(f)][exc_mask_reg[i].iymin : exc_mask_reg[i].iymax + 1, exc_mask_reg[i].ixmin : exc_mask_reg[i].ixmax + 1] = 0.
        globals()['sky_{}'.format(f)] = np.mean(np.true_divide(globals()['ob_dt_{}_c'.format(f)].sum(1),(globals()['ob_dt_{}_c'.format(f)]!= 0.).sum(1)))
    print('하늘 값_' + f + ':', globals()['sky_{}'.format(f)])
print('--------------------------------------')    
    
vmax = 90.
vmin = 15.
print('max_percetile:', vmax)
print('min_percetile:', vmin)
    
# 그림(i필터만)
fig = plt.figure( figsize = (10,4) )
globals()[ 'max_value_{}_temp'.format(f_1) ] = np.percentile( globals()[ 'ob_dt_{}'.format(f_1) ], vmax )
globals()[ 'min_value_{}_temp'.format(f_1) ] = np.percentile( globals()[ 'ob_dt_{}'.format(f_1) ], vmin )
ax = fig.add_subplot(1,2,1)
plt.imshow(globals()['ob_dt_{}'.format(f_1)], cmap = 'gray_r', vmax = globals()['max_value_{}_temp'.format(f_1)], vmin = globals()[ 'min_value_{}_temp'.format(f_1)], origin = 'lower')
plt.plot(globals()['exc_ob_{}'.format(f_1)][1], globals()['exc_ob_{}'.format(f_1)][0], 'o', ms = 2, mec='red', mfc='red', linewidth=1)
plt.grid(color = 'white', ls = '--')
plt.colorbar()
plt.title(f_1 + '_exception_region')

ax = fig.add_subplot(1,2,2)
plt.imshow(globals()['ob_dt_{}'.format(f_1)], cmap = 'gray_r', vmax = globals()['max_value_{}_temp'.format(f_1)], vmin = globals()[ 'min_value_{}_temp'.format(f_1)], origin = 'lower')
for m, o in zip(range(len(exc_mask_reg)), range(len(exc_aperture_reg))):
    ax.add_artist(exc_mask_reg[m].as_artist(facecolor='none', edgecolor='white', linewidth=2))
    ax.add_artist(exc_aperture_reg[o].as_artist(facecolor='none', edgecolor='orange', linewidth=2))  
plt.grid(color = 'white', ls = '--')
plt.colorbar()
plt.title(f_1 + '_exception_region_mask')
plt.show()

### **배경 하늘의 밝기가 제거된 하늘 값 추출**

* 배경 하늘의 밝기가 제거된 하늘 값 추출(skysub)은 은하만의 고유 밝기를 알기 위한 작업입니다. 이를 위해 '원본 이미지'에서 '배경 하늘 값'을 빼며 알고리즘은 다음의 규칙으로 작동합니다.
* 먼저 '원본 이미지'와 '천체마스킹평균 배경하늘 값'을 뺀 것을 우선으로 사용합니다.
* 그런데 그 과정에서 Nan이 발생하면(너무 많이 마스킹하면 Nan 발생), '원본 이미지'와 '5개 최소구역평균 배경하늘 값'을 뺀 것을 사용합니다.

In [ ]:
# 배경 하늘의 밝기가 제거된 하늘 값 추출(skysub)

my_skysub_u = []
my_skysub_g = []
my_skysub_r = []
my_skysub_i = []
my_skysub_z = []

if math.isnan(globals()['sky_{}'.format(f)]) == False:
    for f in filter: # 배경 하늘의 밝기가 제거된 하늘 값 추출시 제외영역의 통계값이 nan 값이 아니면, '천체마스킹평균 배경하늘 값'을 배경 하늘 평균값으로 사용 
        globals()['skysub_{}'.format(f)] = (globals()['ob_dt_{}'.format(f)] - globals()['sky_{}'.format(f)]) # 배경 하늘의 밝기가 제거된 하늘 값 = 원본 값 - 배경 하늘 평균값
        print("배경 하늘의 밝기가 제거된 하늘 값 추출시 제외 영역의 통계값이 nan 값이 아니므로 " + str(f) + " 필터의 천체마스킹평균 배경하늘 값이 이 계산에 사용됨")
else:
    for f in filter: # 배경 하늘의 밝기가 제거된 하늘 값 추출시 제외영역의 통계값이 nan 값이면, '5개 최소구역평균 배경하늘 값' 을  배경 하늘 평균값으로 사용
        globals()['skysub_{}'.format(f)] = (globals()['ob_dt_{}'.format(f)] - globals()['sky_nosie_mean_least5_mean_{}'.format(f)]) # 배경 하늘의 밝기가 제거된 하늘 값 = 원본 값 - 배경 하늘 평균값
        print("배경 하늘의 밝기가 제거된 하늘 값 추출시 제외 영역의 통계값이 nan 값이므로 "+ str(f) + " 필터의 5개 최소구역평균 배경하늘 값이 이 계산에 사용됨")

for f in filter:
    if f == 'u':
        my_skysub_u.append(np.min(globals()['skysub_{}'.format(f)]))
        my_skysub_u.append(np.max(globals()['skysub_{}'.format(f)]))
        my_skysub_u.append(np.mean(globals()['skysub_{}'.format(f)]))
        my_skysub_u.append(np.median(globals()['skysub_{}'.format(f)]))
        my_skysub_u.append(np.std(globals()['skysub_{}'.format(f)]))
    elif f == 'g':
        my_skysub_g.append(np.min(globals()['skysub_{}'.format(f)]))
        my_skysub_g.append(np.max(globals()['skysub_{}'.format(f)]))
        my_skysub_g.append(np.mean(globals()['skysub_{}'.format(f)]))
        my_skysub_g.append(np.median(globals()['skysub_{}'.format(f)]))
        my_skysub_g.append(np.std(globals()['skysub_{}'.format(f)]))
    elif f == 'r':
        my_skysub_r.append(np.min(globals()['skysub_{}'.format(f)]))
        my_skysub_r.append(np.max(globals()['skysub_{}'.format(f)]))
        my_skysub_r.append(np.mean(globals()['skysub_{}'.format(f)]))
        my_skysub_r.append(np.median(globals()['skysub_{}'.format(f)]))
        my_skysub_r.append(np.std(globals()['skysub_{}'.format(f)]))      
    elif f == 'i':
        my_skysub_i.append(np.min(globals()['skysub_{}'.format(f)]))
        my_skysub_i.append(np.max(globals()['skysub_{}'.format(f)]))
        my_skysub_i.append(np.mean(globals()['skysub_{}'.format(f)]))
        my_skysub_i.append(np.median(globals()['skysub_{}'.format(f)]))
        my_skysub_i.append(np.std(globals()['skysub_{}'.format(f)]))
    else:
        my_skysub_z.append(np.min(globals()['skysub_{}'.format(f)]))
        my_skysub_z.append(np.max(globals()['skysub_{}'.format(f)]))
        my_skysub_z.append(np.mean(globals()['skysub_{}'.format(f)]))
        my_skysub_z.append(np.median(globals()['skysub_{}'.format(f)]))
        my_skysub_z.append(np.std(globals()['skysub_{}'.format(f)]))

In [ ]:
# 은하만의 밝기 산출 및 데이터 프레임 변환 및 csv 저장

# 은하만의 밝기 산출 및 데이터 프레임 변환
all_skysub = {
    'min' : [my_skysub_u[0], my_skysub_g[0], my_skysub_r[0], my_skysub_i[0], my_skysub_z[0]], # 각 필터별 하늘의 상대적 세기 최소값
    'max' : [my_skysub_u[1], my_skysub_g[1], my_skysub_r[1], my_skysub_i[1], my_skysub_z[1]], # 각 필터별 하늘의 상대적 세기 최대값
    'mean' : [my_skysub_u[2], my_skysub_g[2], my_skysub_r[2], my_skysub_i[2], my_skysub_z[2]], # 각 필터별 하늘의 상대적 세기 평균값
    'median' : [my_skysub_u[3], my_skysub_g[3], my_skysub_r[3], my_skysub_i[3], my_skysub_z[3]], # 각 필터별 하늘의 상대적 세기 중앙값
    'stdev' : [my_skysub_u[4], my_skysub_g[4], my_skysub_r[4], my_skysub_i[4], my_skysub_z[4]], # 각 필터별 하늘의 상대적 세기 표준편차값
    }
columns = ['min', 'max', 'mean', 'median', 'stdev']
index = ['u', 'g', 'r', 'i', 'z']
skysub_frame = pd.DataFrame(all_skysub, index = index, columns=columns).round(3)
skysub_frame.index.name ='filter'
display(skysub_frame)

# csv 파일 저장
save_file_name = str(ring_num)+'_1_skysub.csv'
skysub_frame.to_csv(save_path + save_file_name, header=True, index=False)

In [ ]:
# 배경 하늘의 밝기가 제거된 하늘 값 추출 그림 시각화 

vmax = 90.
vmin = 15.
print('max_percetile:', vmax)
print('min_percetile:', vmin)

# 그림
fig = plt.figure( figsize = (14,9) )  
n = 1
for f in filter:
    globals()['max_value_{}'.format(f)] = np.percentile(globals()['skysub_{}'.format(f)], vmax)
    globals()['min_value_{}'.format(f)] = np.percentile(globals()['skysub_{}'.format(f)], vmin)
    ax = fig.add_subplot(2,3,n)
    plt.imshow(globals()['skysub_{}'.format(f)], cmap = 'gray_r', vmax = globals()['max_value_{}'.format(f)], vmin = globals()['min_value_{}'.format(f)], origin = 'lower')
    plt.plot(center[1], center[0], '+', ms = 10, mec='red', mfc='red', linewidth=2)
    plt.grid(color = 'white', ls = '--')
    plt.colorbar()
    plt.title(f + '_sky_sub')
    n += 1
plt.show()

## **데이터 다림질(Smoothing)**

### **커털 표준편차와 모드를 사용한 다림질 옵션 조정**

* 데이터 다림질은 잡음과 섞인 이미지의 신호를 정규분포 형태로 부드럽게 바꿔주는 작업입니다.
* 다림질의 옵션에는 가우시안 2D커널의 가우시안 표준편차와 커널 모드가 있습니다. 기본값으로 진행할시 엔터를 누르십시오.
* 가우시안 표준편차가 클수록 등광도선이 더 부드럽고 단순해집니다.
* 가우시안 커널 모드 포함 다림질에 대한 자세한 설명은 다음의 주소에서 확인하십시오. https://docs.astropy.org/en/stable/api/astropy.convolution.Gaussian2DKernel.html

In [ ]:
# 데이터 다림질 및 그 결과 데이터 프레임 변환 csv로 저장하는 함수 정의
def smoothing(kernel_stddev = 0.9, kernel_mode = 'center', x_size = None, y_size = None, sub_u = skysub_u, sub_g = skysub_g, sub_r = skysub_r, sub_i = skysub_i, sub_z = skysub_z):

    # 다림질
    kernel = Gaussian2DKernel(kernel_stddev, mode = kernel_mode, x_size=x_size, y_size=y_size) #  다림질을 위한 가우시안2D 커널 표준편차 및 모드 지정   
    print('가우시안2D 커널 표준편차:', kernel_stddev) #  다림질을 위한 가우시안2D 커널 표준편차
    print('가우시안2D 커널 모드:', kernel_mode) # 다림질을 위한 커널 모드, 커널 종류: 'linear_interp', 'oversample', 'integrate'

    # 결과
    my_smo_u = []
    my_smo_g = []
    my_smo_r = []
    my_smo_i = []
    my_smo_z = []
 
    smo_u = convolve(sub_u, kernel, nan_treatment='interpolate')
    my_smo_u.append(np.min(smo_u))
    my_smo_u.append(np.max(smo_u))
    my_smo_u.append(np.mean(smo_u))
    my_smo_u.append(np.median(smo_u))
    my_smo_u.append(np.std(smo_u))
    
    smo_g = convolve(sub_g, kernel, nan_treatment='interpolate')
    my_smo_g.append(np.min(smo_g))
    my_smo_g.append(np.max(smo_g))
    my_smo_g.append(np.mean(smo_g))
    my_smo_g.append(np.median(smo_g))
    my_smo_g.append(np.std(smo_g))    

    smo_r = convolve(sub_r, kernel, nan_treatment='interpolate')
    my_smo_r.append(np.min(smo_r))
    my_smo_r.append(np.max(smo_r))
    my_smo_r.append(np.mean(smo_r))
    my_smo_r.append(np.median(smo_r))
    my_smo_r.append(np.std(smo_r))    
     
        
    smo_i = convolve(sub_i, kernel, nan_treatment='interpolate')
    my_smo_i.append(np.min(smo_i))
    my_smo_i.append(np.max(smo_i))
    my_smo_i.append(np.mean(smo_i))
    my_smo_i.append(np.median(smo_i))
    my_smo_i.append(np.std(smo_i))
 
    smo_z = convolve(sub_z, kernel, nan_treatment='interpolate')
    my_smo_z.append(np.min(smo_z))
    my_smo_z.append(np.max(smo_z))
    my_smo_z.append(np.mean(smo_z))
    my_smo_z.append(np.median(smo_z))
    my_smo_z.append(np.std(smo_z))  
        
    # 데이터 다림질 값 산출 및 데이터 프레임 변환
    all_smo = {
        'min' : [my_smo_u[0], my_smo_g[0], my_smo_r[0], my_smo_i[0], my_smo_z[0]], # 각 필터별 다림질 최소값
        'max' : [my_smo_u[1], my_smo_g[1], my_smo_r[1], my_smo_i[1], my_smo_z[1]], # 각 필터별 다림질 최대값
        'mean' : [my_smo_u[2], my_smo_g[2], my_smo_r[2], my_smo_i[2], my_smo_z[2]], # 각 필터별 다림질 평균값
        'median' : [my_smo_u[3], my_smo_g[3], my_smo_r[3], my_smo_i[3], my_smo_z[3]], # 각 필터별 다림질 중앙값
        'stdev' : [my_smo_u[4], my_smo_g[4], my_smo_r[4], my_smo_i[4], my_smo_z[4]], # 각 필터별 다림질 표준편차값
        }
    columns = ['min', 'max', 'mean', 'median', 'stdev']
    index = ['u', 'g', 'r', 'i', 'z']
    smo_frame = pd.DataFrame(all_smo, index = index, columns=columns).round(3)
    smo_frame.index.name ='filter'
    display(smo_frame)
    
    # csv 파일 저장
    save_file_name = str(ring_num)+'_2_smo.csv'
    smo_frame.to_csv(save_path + save_file_name, header=True, index=False)
    
    return smo_u, smo_g, smo_r, smo_i, smo_z

In [ ]:
# 데이터 다림질
kn_st = float(input("다림질의 커널 표준편차 값을 입력하세요. 기본값으로 진행할시 엔터를 누르십시오.(기본값: 0.9): ") or 0.9) 
kn_md = str(input("다림질의 커널 모드를 입력하세요. 기본값으로 진행할시 엔터를 누르십시오.(기본값: center, 커널 종류: linear_interp, oversample, integrate): ") or 'center')
smo_u, smo_g, smo_r, smo_i, smo_z = smoothing(kernel_stddev=kn_st, kernel_mode=kn_md)

In [ ]:
# 다림질 그림 시각화

vmax = 90.
vmin = 15.
print('max_percetile:', vmax)
print('min_percetile:', vmin)

# 그림
fig = plt.figure( figsize = (14,9) ) 
n = 1
for f in filter:
    globals()['max_value_{}'.format(f)] = np.percentile(globals()['smo_{}'.format(f)], vmax)
    globals()['min_value_{}'.format(f)] = np.percentile(globals()['smo_{}'.format(f)], vmin)

    ax = fig.add_subplot(2,3,n)
    plt.imshow(globals()['smo_{}'.format(f)], cmap = 'gray_r', vmax = globals()['max_value_{}'.format(f)], vmin = globals()['min_value_{}'.format(f)], origin = 'lower')
    plt.plot(center[1], center[0], '+', ms = 10, mec='red', mfc='red', linewidth=2)
    plt.grid(color = 'white', ls = '--')
    plt.colorbar()
    plt.title(f + '_smoothing')
    n += 1
plt.show()

# **데이터 분석**

## **타원 기하학 산출**

### **이미지 중심좌표 보정 및 등광도선 설정**

* photutils의 EllipseGeometry로 구해진 각 필터당 정확한 이미지 중심 좌표를 산출합니다.

In [ ]:
# photutils의 EllipseGeometry로 구해진 각 필터당 정확한 이미지 중심 좌표

geometry0 = EllipseGeometry(x0=center[1], y0=center[0], sma=20, eps=0.5, pa=-0.5) # EllipseGeometry의 초기값 지정
for f in filter:
    print(f)
    geometry0.find_center(globals()['smo_{}'.format(f)]) # EllipseGeometry를 통해 이미지의 중심 좌표 찾기

* 위에서 산출한 이미지 중심 좌표를 각 필터별로 정확히 입력하여 그 위치를 보정합니다.

In [ ]:
# 위에서 출력된 정확한 이미지 중심 좌표 임력하기

cen_u = [] # u 필터의 정확환 중심 좌표
cen_g = [] # g 필터의 정확환 중심 좌표
cen_r = [] # r 필터의 정확환 중심 좌표
cen_i = [] # i 필터의 정확환 중심 좌표
cen_z = [] # z 필터의 정확환 중심 좌표
for f in filter:
    cen_r1, cen_r2 = map(int, input("쉼표(,)를 이용해 위에서 산출한 %s 필터의 중심 좌표(x,y)를 입력하세요.(예:150,150): "%f).split(',')) 
    globals()['cen_{}'.format(f)].append(cen_r1)
    globals()['cen_{}'.format(f)].append(cen_r2)

* 등광도선의 최소 백분위와 레벨을 조정합니다. 기본값으로 진행할시 엔터를 누르십시오.
* 등광도선의 최소 백분위가 낮을수록 등광도선이 더 자세하게, 레벨은 높을수록 더 많이 등광도선이 그려집니다.
* 은하 내부에 천체가 있는지, 은하 내부가 복잡한지 등 등광도선의 모습을 통해 은하의 상태를 확인합니다.
* 은하 내부에 천체가 있는 이유는, 사실 시선 방향에 있던 별과 겹쳐서 마치 은하 내부에 천체가 있는 것처럼 이미지에 투영되었기 때문입니다. 즉, 은하 내부에는 별이 없습니다.
* 은하 내부가 복잡한 이유는, 은하를 희미한 밝기를 더 많이 포착하기 위해 등광도선을 많이 그렸기 때문입니다. 

In [ ]:
# 등광도선 최소 백분위, 레벨 조정

min_per = int(input("등광도선의 최소 백분위를 입력하세요. 기본값으로 진행할시 엔터를 누르십시오.(기본값: 89): ") or 89)
step_num = int(input("등광도선의 레벨을 입력하세요. 기본값으로 진행할시 엔터를 누르십시오.(기본값: 25): ") or 25)
max_per = 99.9
print('최고 백분위:', min_per)
print('최소 백분위:', max_per)
print('레벨:', step_num) 
for f in filter:
    globals()['levels_{}'.format(f)] = np.linspace(np.percentile(globals()['smo_{}'.format(f)], min_per), np.percentile(globals()['smo_{}'.format(f)], max_per), step_num)

In [ ]:
# 이미지 등광도선 확인
def image_contour_identification():
    vmax = 90.
    vmin = 15.

    for f in filter:
        globals()['max_value_{}_temp'.format(f)] = np.percentile( globals()['smo_{}'.format(f)], vmax)
        globals()['min_value_{}_temp'.format(f)] = np.percentile( globals()['smo_{}'.format(f)], vmin)
        globals()['max_{}'.format(f)] = np.max(globals()['smo_{}'.format(f)])

    # 그림
    fig = plt.figure( figsize = (17, 8) )

    ax1 = fig.add_subplot(2,4,1)
    im = plt.imshow(globals()['smo_{}'.format(filter[1])], cmap = 'gray_r', vmax = globals()['max_value_{}_temp'.format(filter[1])], vmin = globals()[ 'min_value_{}_temp'.format(filter[1])], origin = 'lower') #interpolation='nearest'
    plt.colorbar(im)
    ct = plt.contour(globals()['smo_{}'.format(filter[1])], levels = globals()['levels_{}'.format(filter[1])], origin = 'lower', colors = 'aqua', linewidths = 1.0)
    plt.plot(globals()['cen_{}'.format(filter[1])][0], globals()['cen_{}'.format(filter[1])][1], '+', ms = 10, mec='red', mfc='red', linewidth=2)
    plt.grid(color = 'white', ls = '--')
    plt.title(filter[1] + '_reg')

    ax2 = fig.add_subplot(2,4,2)
    ct = plt.contour(globals()['smo_{}'.format(filter[1])], levels = globals()['levels_{}'.format(filter[1])], origin = 'lower', linewidths = 0.5, colors = 'black') #colors= colors,
    #plt.xlim(globals()['cen_{}'.format(filter[1])][0]-50, globals()['cen_{}'.format(filter[1])][0]+50) # 이미지 확대를 위해 중심 기준 y축 방향 +-50 처리함
    #plt.ylim(globals()['cen_{}'.format(filter[1])][1]-50, globals()['cen_{}'.format(filter[1])][1]+50) # 이미지 확대를 위해 중심 기준 y축 방향 +-50 처리함
    plt.plot(globals()['cen_{}'.format(filter[1])][0], globals()['cen_{}'.format(filter[1])][1], '+', ms = 10, mec='red', mfc='red', linewidth=2)
    plt.grid(color = 'white', ls = '--')
    plt.title(filter[1] + '_reg, min_per: ' + str(min_per) + ', max_per: ' + str(max_per) + ', step: ' + str(step_num))
    plt.show()

    ax3 = fig.add_subplot(2,4,3)
    im = plt.imshow(globals()['smo_{}'.format(filter[2])], cmap = 'gray_r', vmax = globals()['max_value_{}_temp'.format(filter[2])], vmin = globals()[ 'min_value_{}_temp'.format(filter[2])], origin = 'lower') #interpolation='nearest'
    plt.colorbar(im)
    ct = plt.contour(globals()['smo_{}'.format(filter[2])], levels = globals()['levels_{}'.format(filter[2])], origin = 'lower', colors = 'aqua', linewidths = 1.0)
    plt.plot(globals()['cen_{}'.format(filter[2])][0], globals()['cen_{}'.format(filter[2])][1], '+', ms = 10, mec='red', mfc='red', linewidth=2)
    plt.grid(color = 'white', ls = '--')
    plt.title(filter[2] + '_reg')

    ax4 = fig.add_subplot(2,4,4)
    ct = plt.contour(globals()['smo_{}'.format(filter[2])], levels = globals()['levels_{}'.format(filter[2])], origin = 'lower', linewidths = 0.5, colors = 'black') #colors= colors,
    #plt.xlim(globals()['cen_{}'.format(filter[2])][0]-50, globals()['cen_{}'.format(filter[2])][0]+50) # 이미지 확대를 위해 중심 기준 y축 방향 +-50 처리함
    #plt.ylim(globals()['cen_{}'.format(filter[2])][1]-50, globals()['cen_{}'.format(filter[2])][1]+50) # 이미지 확대를 위해 중심 기준 y축 방향 +-50 처리함
    plt.plot( globals()['cen_{}'.format(filter[2])][0],  globals()['cen_{}'.format(filter[2])][1], '+', ms = 10, mec='red', mfc='red', linewidth=2)
    plt.grid(color = 'white', ls = '--')
    plt.title(filter[2] + '_reg, min_per: ' + str(min_per) + ', max_per: ' + str(max_per) + ', step: ' + str(step_num))
    plt.show()

    ax5 = fig.add_subplot(2,4,5)
    im = plt.imshow(globals()['smo_{}'.format(filter[3])], cmap = 'gray_r', vmax = globals()['max_value_{}_temp'.format(filter[3])], vmin = globals()[ 'min_value_{}_temp'.format(filter[3])], origin = 'lower') #interpolation='nearest'
    plt.colorbar(im)
    ct = plt.contour(globals()['smo_{}'.format(filter[3])], levels = globals()['levels_{}'.format(filter[3])], origin = 'lower', colors = 'aqua', linewidths = 1.0)
    plt.plot( globals()['cen_{}'.format(filter[3])][0], globals()['cen_{}'.format(filter[3])][1], '+', ms = 10, mec='red', mfc='red', linewidth=2)
    plt.grid(color = 'white', ls = '--')
    plt.title(filter[3] + '_reg')

    ax6 = fig.add_subplot(2,4,6)
    ct = plt.contour(globals()['smo_{}'.format(filter[3])], levels = globals()['levels_{}'.format(filter[3])], origin = 'lower', linewidths = 0.5, colors = 'black') #colors= colors,
    #plt.xlim(globals()['cen_{}'.format(filter[3])][0]-50, globals()['cen_{}'.format(filter[3])][0]+50) # 이미지 확대를 위해 중심 기준 y축 방향 +-50 처리함
    #plt.ylim(globals()['cen_{}'.format(filter[3])][1]-50, globals()['cen_{}'.format(filter[3])][1]+50) # 이미지 확대를 위해 중심 기준 y축 방향 +-50 처리함
    plt.plot(globals()['cen_{}'.format(filter[3])][0], globals()['cen_{}'.format(filter[3])][1], '+', ms = 10, mec='red', mfc='red', linewidth=2)
    plt.grid(color = 'white', ls = '--')
    plt.title(filter[3] + '_reg, min_per: ' + str(min_per) + ', max_per: ' + str(max_per) + ', step: ' + str(step_num))
    plt.show()

    ax7 = fig.add_subplot(2,4,7)
    im = plt.imshow(globals()['smo_{}'.format(filter[4])], cmap = 'gray_r', vmax = globals()['max_value_{}_temp'.format(filter[4])], vmin = globals()[ 'min_value_{}_temp'.format(filter[4])], origin = 'lower') #interpolation='nearest'
    plt.colorbar(im)
    ct = plt.contour(globals()['smo_{}'.format(filter[4])], levels = globals()['levels_{}'.format(filter[4])], origin = 'lower', colors = 'aqua', linewidths = 1.0)
    plt.plot( globals()['cen_{}'.format(filter[4])][0], globals()['cen_{}'.format(filter[4])][1], '+', ms = 10, mec='red', mfc='red', linewidth=2)
    plt.grid(color = 'white', ls = '--')
    plt.title(filter[4] + '_reg')

    ax8 = fig.add_subplot(2,4,8)
    ct = plt.contour(globals()['smo_{}'.format(filter[4])], levels = globals()['levels_{}'.format(filter[4])], origin = 'lower', linewidths = 0.5, colors = 'black') #colors= colors,
    #plt.xlim(globals()['cen_{}'.format(filter[4])][0]-50, globals()['cen_{}'.format(filter[4])][0]+50) # 이미지 확대를 위해 중심 기준 y축 방향 +-50 처리함
    #plt.ylim(globals()['cen_{}'.format(filter[4])][1]-50, globals()['cen_{}'.format(filter[4])][1]+50) # 이미지 확대를 위해 중심 기준 y축 방향 +-50 처리함
    plt.plot(globals()['cen_{}'.format(filter[4])][0], globals()['cen_{}'.format(filter[4])][1], '+', ms = 10, mec='red', mfc='red', linewidth=2)
    plt.grid(color = 'white', ls = '--')
    plt.title(filter[4] + '_reg, min_per: ' + str(min_per) + ', max_per: ' + str(max_per) + ', step: ' + str(step_num))
    plt.show()
    
image_contour_identification()

In [ ]:
# 이미지 저장

save_file_name = str(ring_num)+'_0_image_contour_identification.jpg'
plt.savefig(save_path + save_file_name)

### **내부 천체 마스킹 및 다림질 조정**

* 위 등광도선 작업에서 내부에 천체가 존재한다면, 그 천체를 마스킹 하고 다시 다림질을 합니다.
* 내부 천체 마스킹 및 다림질 조정이 필요하다면 -> y를 입력하고 -> 마스킹 좌표, 반경을 입력합니다. -> 그 다음으로 다림질 옵션을 조정하며, 기본값으로 진행할시 엔터를 누릅니다. 천체 마스킹시 가우시안 표준편차만 1.5~2로 조정하는게 좋습니다.
* 내부 천체 마스킹 및 다림질 조정이 필요없다면 -> n을 입력합니다.

In [ ]:
# 추가로 천체 마스킹 및 재다림질 조정

select = str(input('추가로 마스킹 할 천체 영역이 있는가요? y, n: '))
print('-------------------------------------------------')    
if select == 'y':
    mask_ob_cor=[]
    mask_ob_rad=[]
    n = 1
    try:
        for i, j, r in zip(range(1,40,2), range(2,41,2), range(1,20,1)):
            [globals()['num_{}'.format(i)], globals()['num_{}'.format(j)], globals()['rad_{}'.format(r)]] = map(float, input("추가 천체 제거를 위해, 콤마(,)를 이용해 %d번째 마스크 하려는 영역의 중심 좌표(x,y)와 반경(r)을 0 보다 큰 수로 순서대로 입력하세요.(예:25,25,5) 그만 두려면 아무 값이나 입력하세요.: "%n).split(','))
            mask_ob_cor.append([ globals()['num_{}'.format(i)],globals()['num_{}'.format(j)]])
            mask_ob_rad.append([ globals()['rad_{}'.format(r)]])
            n += 1
    except:
        pass

    # 추가 천체 마스킹 
    mask_reg = []
    aperture_reg = []
    n = 1
    for k, r in zip(mask_ob_cor, mask_ob_rad):
        center_1 = PixCoord(k[0], k[1])
        #print('%d_[x,y,r]:'%n, k[0], k[1], r[0])
        aperture = CirclePixelRegion(center_1, r[0])
        mask = aperture.to_mask(mode = 'center')
        aperture_reg.append(aperture)
        mask_reg.append(mask.bbox)
        n += 1
    
    for f in filter:
        globals()['skysub_{}_c'.format(f)] = globals()['skysub_{}'.format(f)].copy()
        for i in range(len(mask_reg)): 
            globals()['skysub_{}_c'.format(f)][mask_reg[i].iymin : mask_reg[i].iymax + 1, mask_reg[i].ixmin : mask_reg[i].ixmax + 1] = np.nan
            
    print('-------------------------------------------------')         
    # 데이터 다림질
    kn_st1 = float(input("다림질의 커널 표준편차 값을 입력하세요. 기본값으로 진행할시 엔터를 누르십시오.(기본값: 0.9): ") or 0.9) 
    kn_md1 = str(input("다림질의 커널 모드를 입력하세요. 기본값으로 진행할시 엔터를 누르십시오.(기본값: center): ") or 'center')
    smo_u, smo_g, smo_r, smo_i, smo_z = smoothing(kernel_stddev=kn_st1, kernel_mode=kn_md1, x_size = (10*int(r[0]))+1, y_size = (10*int(r[0]))+1, sub_u=skysub_u_c, sub_g=skysub_g_c, sub_r=skysub_r_c, sub_i=skysub_i_c, sub_z=skysub_z_c)
    image_contour_identification()
        
    print("변경 완료: 다음 단계로 넘어가 은하 위에 타원을 그리기 위한 작업을 진행하십시오.")
    
else:
    print("다음 단계로 넘어가 은하 위에 타원을 그리기 위한 작업을 진행하십시오.")

### **타원 기하학 산출**

* 타원 기하학을 산출하려면 은하 위에 타원을 그려야 합니다.
* 은하 위에 그려질 타원의 장축 반경(픽셀)을 마우스 왼쪽 버튼으로 클릭하세요.(오른쪽 버튼: 선택 취소)
* ※ 유의사항: 1. 오직 목표 은하만 포착할 수 있게끔 장축 반경을 고려하세요. 2. 여러 곳 클릭해도 마지막 최근 값만 장축 반경 계산에 적용됩니다.

In [ ]:
# 이미지에 좌표 클릭하는 함수 정의
def add_point(event):
    if event.inaxes != ax:
        return
    # button 1: 마우스 좌클릭 시 입력값 획득
    if event.button == 1:
        x = event.xdata
        y = event.ydata
        xdata.append(x)
        ydata.append(y)
        line.set_data(xdata, ydata)
        plt.plot([cen[0], x],  [cen[1], y], linewidth=2) # color = 'yellow',
        plt.draw()
        tx = 'select, x=%.1f, y=%.1f' % (np.around(x, decimals = 1), np.around(y, decimals = 1))
        text.set_text(tx)
    # button 3: 마우스 우클릭 시 기존 입력값 삭제
    if event.button == 3:
        xdata.pop()
        ydata.pop()
        line.set_data(xdata, ydata)
        plt.draw()
        tx = 'delete, x=%.1f, y=%.1f' % (np.around(x, decimals = 1), np.around(y, decimals = 1))
        text.set_text(tx)
    # 마우스 중간버튼 클릭 시 종료하기
    if event.button == 2:
        plt.disconnect(cid)
        plt.close()

In [ ]:
#이미지에 좌표 클릭시 포인팅 및 좌표 출력

f_1 = 'i'

print('은하 이미지에 그려질 타원의 장축 반경을 마우스 왼쪽 버튼으로 클릭하세요.(오른쪽 버튼: 선택 취소)')
print('※유의사항: 1. 오직 목표 은하만 포착할 수 있게끔 장축 반경을 고려하세요. 2. 여러 곳을 클릭해도 마지막 최근 값만 장축 반경 계산에 적용됩니다.')
print( '-----------------------------------------' )
xdata = [0]; ydata = [0]

# 모든 필터 중심 좌표
cen = globals()['cen_{}'.format(f_1)]

vmax = 90.
vmin = 15.

# 그림
fig = plt.figure( figsize = (5,5) )
globals()['max_value_{}_temp'.format(f_1)] = np.percentile( globals()['smo_{}'.format(f_1)], vmax )
globals()['min_value_{}_temp'.format(f_1)] = np.percentile( globals()['smo_{}'.format(f_1)], vmin )

globals()['max_{}'.format(f_1)] = np.max(globals()['smo_{}'.format(f_1)])

ax = fig.add_subplot(1,1,1)
im = plt.imshow(globals()['smo_{}'.format(f_1)], cmap = 'gray_r', vmax = globals()['max_value_{}_temp'.format(f_1)], vmin = globals()[ 'min_value_{}_temp'.format(f_1)], origin = 'lower', interpolation='nearest') #interpolation='nearest'
plt.colorbar(im)
ct = plt.contour(globals()['smo_{}'.format(f_1)], levels = globals()['levels_{}'.format(f_1)], origin = 'lower', colors = 'aqua', linewidths = 1.0)
plt.plot(cen[0], cen[1], '+', ms = 10, mec='red', mfc='red', linewidth=2)
plt.grid(color = 'white', ls = '--')
line, = ax.plot(xdata, ydata, '*', ms = 10, mec='yellow', mfc='yellow', linewidth=2)
cid = plt.connect('button_press_event', add_point)
text = ax.text(70,330, "", va="bottom", ha="left")
plt.title(f_1 + '_semi-major axis point')

plt.show()

In [ ]:
# 이미지 저장

save_file_name = str(ring_num)+'_1_selcet_coordinate.jpg'
plt.savefig(save_path + save_file_name)

* 은하 위에 그려질 타원의 단축 반경(픽셀)을 적절히 입력합니다.

In [ ]:
# 단축 반경 입력

xdata = np.around(xdata,decimals = 1)
ydata = np.around(ydata,decimals = 1)
# 기본 (0,0) 값 삭제
if (xdata[0], ydata[0]) == (0,0):
    xdata = np.delete(xdata, 0)
    ydata = np.delete(ydata, 0)
    print('마우스 클릭한 장축 x 좌표 모음, y좌표 모음(삭제 값은 미반영): ', xdata, ydata)
else:
    print('마우스 클릭한 장축 x 좌표 모음, y좌표 모음(삭제 값은 미반영): ', xdata, ydata)
        
cor = [int(xdata[-1]), int(ydata[-1])] # 가장 최근인 마지막으로 클릭한 타원의 장축반경 x,y 좌표만 적용
major_ax = math.sqrt((cen[0] - cor[0])**2 + (cen[1] - cor[1])**2)
print('장축 반경 거리(픽셀):', major_ax)
d = float(input('은하 이미지에 그려질 타원의 단축 반경을 입력하세요.(예:75): '))

* 산출된 타원이 은하에 맞게 그려졌는지 확인합니다. 적절하지 않다면 장축 혹은 단축 반경을 조정합니다. 

In [ ]:
# y = ax + b 일차함수로 장축, 단축 반경 계산

a = (cen[1]-cor[1])/(cen[0]-cor[0])
if abs(a) == 0:
    a = 0.000000000001
else:
    a = a
b = cen[1] - a*cen[0]

# y1 = -x/a + b1 -> y = ax + b와 수직인 일차함수
b1 = cen[1] + (cen[0]/a)

# 타원 기하학 계산
minor_ax = d
major_ax = math.sqrt((cen[0]-cor[0])**2+(cen[1]-cor[1])**2)

ellipse = (major_ax - minor_ax)/major_ax
print('타원율:', ellipse)
position_angle_rad = math.atan((cen[1]-cor[1])/(cen[0]-cor[0]))
position_angle_deg = (position_angle_rad)/np.pi*180
print('위치각(deg):', position_angle_deg)
print('위치각(radian):', position_angle_rad)

vmax = 90.
vmin = 15.
print('--------------------------------------')
print('max_percetile:', vmax)
print('min_percetile:', vmin)

# 그림
fig = plt.figure( figsize = (18,9) )
n = 1
for f in filter:
    globals()[ 'geometry_{}'.format(f)] = EllipseGeometry(x0=globals()[ 'cen_{}'.format(f)][0], y0=globals()[ 'cen_{}'.format(f)][1], sma=major_ax, eps=ellipse, pa=position_angle_rad,  astep = 0.1, linear_growth=True) # astep=0,1, fix_center=True, linear_growth=True
    globals()[ 'aper_{}'.format(f)] = EllipticalAperture((globals()[ 'geometry_{}'.format(f)].x0, globals()[ 'geometry_{}'.format(f)].y0), globals()[ 'geometry_{}'.format(f)].sma, globals()[ 'geometry_{}'.format(f)].sma*(1 - globals()[ 'geometry_{}'.format(f)].eps), globals()[ 'geometry_{}'.format(f)].pa)

    globals()[ 'max_value_{}_temp'.format(f) ] = np.percentile( globals()[ 'smo_{}'.format(f) ], vmax )
    globals()[ 'min_value_{}_temp'.format(f) ] = np.percentile( globals()[ 'smo_{}'.format(f) ], vmin )
    ax = fig.add_subplot(2,3,n)
    im = plt.imshow(globals()['smo_{}'.format(f)], cmap = 'gray_r', vmax = globals()['max_value_{}'.format(f)], vmin = globals()[ 'min_value_{}'.format(f)], origin = 'lower') #interpolation = 'nearest'
    ct = plt.contour(globals()['smo_{}'.format(f)], levels = globals()['levels_{}'.format(f_1)], origin = 'lower', colors = 'aqua', linewidths = 1)
    globals()[ 'aper_{}'.format(f)].plot(color = 'greenyellow', linewidth = 6, linestyle = 'dashed')
    plt.grid(color = 'white', ls = '--')
    plt.colorbar(im)
    plt.title(f + '_geometry, major: ' + str(int(major_ax)) + ', minor: ' + str(int(minor_ax)))
    n += 1
plt.show()

In [ ]:
# 이미지 저장

save_file_name = str(ring_num)+'_2_ellipsegeometry_contour.jpg'
plt.savefig(save_path + save_file_name)

## **등광도 분석**

### **등광도 맞춤 분석**

* 은하 특징을 도출하기 위해서는 광도 윤곽, 타원율, 위치각을 분석해내야 하며, 이를 위한 작업으로 등광도 맞춤 분석이 필요합니다. 
* 최초로(처음으로) 하는 등광도 맞춤 분석일 경우, 등광도 맞춤 분석의 옵션을 변경하지 않고 시작합니다(n 입력). 기본값으로 진행하므로 엔터를 누릅니다.
* 등광도 맞춤 분석 후 실제와 분석된 등광도선의 불일치가 심한 경우 등광도 맞춤 분석을 다시 합니다.(y입력). 구체적인 순서는 다음과 같습니다.
* 먼저 다림질의 옵션을 조정합니다. 가우시안 표준편차만 1.5~2로 조정하는게 좋습니다.
*  그 다음으로 등광도 맞춤 분석의 옵션은 장반경 분석 간격(step)을 0.5로, 1픽셀당 실제 거리를 0.198로, nclip은 10~30 중 적절한 값을, fflag은 0.3을 입력합니다.

In [ ]:
# Isolist 정보 추출

iso_select1 = str(input('등광도 맞춤 옵션을 변경 하시겠습니까? 최초로(처음으로) 하는 등광도 맞춤 분석일 경우 옵션 변경을 하지 않습니다.(n입력) y, n: '))
print('---------------------------------------------------')  
if iso_select1 == 'y':
    iso_select2 = str(input('앞서 천체마스킹을 하였나요? y, n: '))
    print('---------------------------------------------------')  
    if iso_select2 == 'n':
    # 데이터 다림질
        kn_st2 = float(input("다림질의 커널 표준편차 값을 입력하세요. 기본값으로 진행할시 엔터를 누르십시오.(기본값: 0.9): ") or 0.9) 
        kn_md2 = str(input("다림질의 커널 모드를 입력하세요. 기본값으로 진행할시 엔터를 누르십시오.(기본값: center): ") or 'center')
        smo_u, smo_g, smo_r, smo_i, smo_z = smooting(kernel_stddev=kn_st2, kernel_mode=kn_md2, x_size=(10*int(r[0]))+1, y_size=(10*int(r[0]))+1, sub_u=skysub_u_c, sub_g=skysub_g_c, sub_r=skysub_r_c, sub_i=skysub_i_c, sub_z=skysub_z_c)
        image_contour_identification()
    elif iso_select2 == 'y':
        pass
    iso_step = float(input("등광도 맞춤 옵션 중 장반경 분석 간격(step)을 입력 하세요. 기본값으로 진행할시 엔터를 누르십시오.(기본값: 1): ") or 1)
    iso_dist = float(input("등광도 맞춤 옵션 중 1픽셀 당 초(arcsec)를 입력 하세요. 기본값으로 진행할시 엔터를 누르십시오.(기본값: 0.396, 주의:step과 정비례로 입력): ") or 0.396)
    iso_nclip = int(input("등광도 맞춤 옵션 중 nclip 값을 입력 하세요. 기본값으로 진행할시 엔터를 누르십시오.(기본값: 0, 주의:자연수 입력): ") or 0.0)
    iso_fflag = float(input("등광도 맞춤 옵션 중 fflag 값을 입력 하세요. 기본값으로 진행할시 엔터를 누르십시오.(기본값: 0.7): ") or 0.7)
else:
    iso_step = 1
    iso_dist = 0.396
    iso_nclip = 0
    iso_fflag = 0.7      
    print("장반경 분석 간격(step, 기본값 1) :", iso_step)
    print("1픽셀당 초(arcsec, 기본값 0.396) :", iso_dist)
    print("nclip(기본값 0) :", iso_nclip)
    print("fflag(기본값 0.7) :", iso_fflag)

for f in filter:
    globals()['r_pix_{}'.format(f)] = [] # 각 필터별 픽셀 기준 등광도 맞춤 분석된 장반경 값 모음
    globals()['mu_{}'.format(f)] = [] # 각 필터별 등광도 맞춤 분석된 광도 값 모음
    globals()['ep_{}'.format(f)] = [] # 각 필터별 등광도 맞춤 분석된 타원율 값 모음
    globals()['ep_err_{}'.format(f)] = [] # 각 필터별 등광도 맞춤 분석된 타원율 에러 값 모음
    globals()['pa_{}'.format(f)] = [] # 각 필터별 등광도 맞춤 분석된 위치각 값 모음
    globals()['pa_err_{}'.format(f)] = [] # 각 필터별 등광도 맞춤 분석된 위치각 에러 값 모음 
    
for f in filter:
    try:
        globals()['ellipse_{}'.format(f)] = Ellipse(globals()['smo_{}'.format(f)], geometry=globals()['geometry_{}'.format(f)])
        globals()['isolist_{}'.format(f)] = globals()['ellipse_{}'.format(f)].fit_image(minsma = 0, step = iso_step, linear = True, maxsma = major_ax, fix_center = True, nclip = iso_nclip, fflag = iso_fflag) 
        globals()['intens_{}'.format(f)] = -2.5*np.log10(globals()['isolist_{}'.format(f)].intens) 
        
        globals()['iso_{}_c'.format(f)] = globals()['isolist_{}'.format(f)].pa.copy()

        # IRAF 기준 타원 위치각 설명과 동일하게 변환(+Y축을 기준으로 +90 ~ -90로 타원 위치각이 분포)
        globals()['iso_{}_c_deg'.format(f)] = globals()['iso_{}_c'.format(f)]/np.pi*180. # 라디안(radian)을 도(degree)로 변환
        globals()['iso_{}_c_deg'.format(f)][globals()['iso_{}_c_deg'.format(f)] < 90] = globals()['iso_{}_c_deg'.format(f)][globals()['iso_{}_c_deg'.format(f)] < 90] + 180 # 90 도 미만에 +180를 하여, 모든 타원 위치각이 180 ~ 270도로 변환
        globals()['pa_{}_deg'.format(f)] = globals()['iso_{}_c_deg'.format(f)] - 90. # 모든 각도에 -90도를 하여, 0 ~ 180 도로 분포하도록 함. 즉, +Y축을 기준으로 +90 ~ -90로 타원 위치각이 분포
        
        globals()['pa_err_{}_deg'.format(f)]= globals()['isolist_{}'.format(f)].pa_err/np.pi*180. # 라디안을 도(degree)로 변환
        globals()['mu_{}'.format(f)].append(np.around(globals()['intens_{}'.format(f)], 3)) # 각 필터별 등광도 맞춤 분석된 광도 값(소수점 3자리 까지만)
        globals()['ep_{}'.format(f)].append(np.around(globals()['isolist_{}'.format(f)].eps, 3)) # 각 필터별 등광도 맞춤 분석된 타원율 값(소수점 3자리 까지만)
        globals()['ep_err_{}'.format(f)].append(np.around(globals()['isolist_{}'.format(f)].ellip_err, 3)) # 각 필터별 등광도 맞춤 분석된 타원율 에러 값(소수점 3자리 까지만)
        globals()['pa_{}'.format(f)].append(np.around(globals()['pa_{}_deg'.format(f)], 3)) # 각 필터별 등광도 맞춤 분석된 위치각 값(소수점 3자리 까지만)
        globals()['pa_err_{}'.format(f)].append(np.around(globals()['pa_err_{}_deg'.format(f)], 3)) # 각 필터별 등광도 맞춤 분석된 위치각 에러 값(소수점 3자리 까지만)
        
    except:
        pass
    
for f in filter:
    for p in range(len(globals()['isolist_{}'.format(f)].sma)):
        globals()['r_pix_{}'.format(f)].append(p)
    globals()['r_arc_{}'.format(f)] = np.around(np.asarray(globals()['r_pix_{}'.format(f)])*iso_dist, 2) 
    # iso_step = 1 이면 iso_dist = 0.396 / iso_step = 0.5 이면, iso_setp = 0.198 로 변화 필요

 ### **등광도 지도 작성(Isophotal contour map)**

* 각 필터별 등광도 지도를 통해 실제 등광도선과 분석된 등광도선이 일치하는지 확인합니다.
* 불일치가 심하다면 은하 내부에 천체가 존재하거나 내부가 복잡하여 이상값을 유발하였기 때문입니다.
* 은하 내부에 천체가 존재한다면, 천체마스킹 작업을 합니다.
* 은하 내부가 복잡하다면, 등광도 맞춤 분석에서 옵션을 조정합니다.
* 은하 내부에 천체가 있는 이유는, 사실 시선 방향에 있던 별과 겹쳐서 마치 은하 내부에 천체가 있는 것처럼 이미지에 투영되었기 때문입니다. 즉, 은하 내부에는 별이 없습니다.
* 은하 내부가 복잡한 이유는, 은하를 희미한 밝기를 더 많이 포착하기 위해 등광도선을 많이 그렸기 때문입니다. 

* 등광도 맞춤 분석 결과를 토대로 등광도 관련 밝기 강도, 타원율, 위치각 특징을 분석함으로써 은하의 특징을 파악합니다.

In [ ]:
# 분석된 등광도곡선 그리기

for f in filter:
    min_pix = 10 # 분석된 등광도 곡선 최소 픽셀 지점(시작점)
    globals()['max_pix_{}'.format(f)] = int(major_ax) # 등광도 곡선 최대 픽셀 지점(끝점) -> 위에서 입력한 장반경 값과 같음
    step = 10

vmax = 90.
vmin = 15.
print('max_percetile', vmax)
print('min_percetile:', vmin)            

# 그림
fig = plt.figure( figsize = (14,9) )
n = 1
for f in filter :
    globals()['max_value_{}_temp'.format(f)] = np.percentile(globals()['smo_{}'.format(f)], vmax)
    globals()['min_value_{}_temp'.format(f)] = np.percentile(globals()['smo_{}'.format(f)], vmin)   
    ax = fig.add_subplot(2,3,n)
    ct = plt.contour(globals()['smo_{}'.format(f)], levels = globals()['levels_{}'.format(f)] , origin = 'lower', colors= 'black', linewidths = 0.5)
    #plt.xlim(globals()['cen_{}'.format(f)][0]-0, globals()['cen_{}'.format(f)][0]+0) # 이미지 확대를 위해 중심 기준 x축 방향 +-50 처리함
    #plt.ylim(globals()['cen_{}'.format(f)][1]-0, globals()['cen_{}'.format(f)][1]+0) # 이미지 확대를 위해 중심 기준 y축 방향 +-50 처리함
    ax.grid(color = 'white', ls = '--')
    ax.set_title(f + '_isophotal contour map')
    smas = np.linspace(min_pix, globals()['max_pix_{}'.format(f)], step)
    try:
        for sma in smas:
            iso = globals()['isolist_{}'.format(f)].get_closest(sma)
            x, y, = iso.sampled_coordinates()
            ax.plot(x, y, ls = '--', color = 'red', lw = 0.5)
    except:
        pass
    n += 1
plt.show()

In [ ]:
# 이미지 저장

save_file_name = str(ring_num)+'_3_isophote_contour.jpg'
plt.savefig(save_path + save_file_name)

### **등광도 관련 광도 윤곽(Luminosity Profile), 타원율(Ellipticity), 위치각(Position Angle) 특징 분석**

* 각 필터별로 분석된 다양한 등광도 맞춤 분석의 물리값을 확인합니다.

In [ ]:
# 등광도 맞춤 분석 물리값 표

display('isolist_u_table', isolist_u.to_table())
display('isolist_g_table', isolist_i.to_table())
display('isolist_r_table', isolist_i.to_table())
display('isolist_i_table', isolist_i.to_table())
display('isolist_z_table', isolist_z.to_table())

In [ ]:
# 등광도 맞춤 분석 물리값 데이터 프레임 변환

for f in filter:
    try:
        columns = ['R"', 'Mu', 'Ellipticity', 'Ellip_error', 'PA', 'PA_error']
        #globals()['df_r_{}'.format(f)] = pd.DataFrame(globals()['sma_{}'.format(f)]).transpose()
        globals()['df_r_{}'.format(f)] = pd.DataFrame(globals()['r_arc_{}'.format(f)]) # 각 필터별 등광도 분석된 장반경
        globals()['df_mu_{}'.format(f)] = pd.DataFrame(globals()['mu_{}'.format(f)]).transpose() # 각 필터별 등광도 분석된 광도 값
        globals()['df_ep_{}'.format(f)] = pd.DataFrame(globals()['ep_{}'.format(f)]).transpose() # 각 필터별 등광도 분석된 타원율 값
        globals()['df_ep_err_{}'.format(f)] = pd.DataFrame(globals()['ep_err_{}'.format(f)]).transpose() # 각 필터별 등광도 분석된 타원율 에러 값
        globals()['df_pa_{}'.format(f)] = pd.DataFrame(globals()['pa_{}'.format(f)]).transpose() # 각 필터별 등광도 분석된 위치각 값
        globals()['df_pa_err_{}'.format(f)] = pd.DataFrame(globals()['pa_err_{}'.format(f)]).transpose() # 각 필터별 등광도 분석된 위치각 에러 값
    
        globals()['df_my_eps_{}'.format(f)] = pd.concat([globals()['df_r_{}'.format(f)], globals()['df_mu_{}'.format(f)], globals()['df_ep_{}'.format(f)], globals()['df_ep_err_{}'.format(f)], globals()['df_pa_{}'.format(f)], globals()['df_pa_err_{}'.format(f)]], axis = 1)
        globals()['df_my_eps_{}'.format(f)].columns = columns
    except:
        pass

In [ ]:
# csv 파일 저장

df_all = []
for f in filter:
    cc = globals()['df_my_eps_{}'.format(f)].rename(columns = lambda x: f +'_' + x)
    df_all.append(cc)
    save_file_name1 = str(ring_num)+'_4_'+ f + '_eps__result.csv'
    globals()['df_my_eps_{}'.format(f)].to_csv(save_path + save_file_name1, header=True, index=False)

save_file_name2 = str(ring_num)+'_eps__result.csv' 
df_all_concat = pd.concat(df_all, axis=1)
df_all_concat.to_csv(save_path + save_file_name2, header=True, index=False)   
display(df_all_concat)

* 에러 값을 포함한 광도 윤곽의 광도와 타원율, 위치각의 변화를 분석함으로써 은하의 특징을 살펴봅니다.

In [ ]:
# 그래프 시각화 에러 값 포함

plt.ion()
plt.figure(figsize=(6, 3.5), dpi=300)
for i,j,k,f in zip(range(1,17,3), range(2,18,3), range(3,19,3), filter):
    plt.subplot(5,3,i)
    plt.plot(globals()['df_my_eps_{}'.format(f)]['R"'][10:], globals()['df_my_eps_{}'.format(f)]['Mu'][10:], 'r^-', ms=0.5, linewidth=0.2, label= f)
    plt.xlabel('Major_R(arcsec)', fontsize = 3 ,labelpad = 1) 
    plt.ylabel('Magnitude', fontsize = 3, labelpad = 1)
    plt.xticks()
    plt.yticks()
    plt.tick_params(axis = 'x', direction = 'out', length = 1.5, width = 0.5, labelsize = 3, pad = 1)
    plt.tick_params(axis = 'y', direction = 'out', length = 1.5, width = 0.5, labelsize = 3, pad = 1)
    plt.gca().invert_yaxis()
    plt.legend(loc = 'best', fontsize = 3.5)

    plt.subplot(5,3,j)
    plt.errorbar(globals()['df_my_eps_{}'.format(f)]['R"'][10:], globals()['df_my_eps_{}'.format(f)]['Ellipticity'][10:], globals()['df_my_eps_{}'.format(f)]['Ellip_error'][10:], fmt='r^-', ms=0.5, linewidth=0.2, elinewidth=0.2, ecolor='black', capsize=1, capthick=0.5, label= f)
    plt.xlabel('Major_R(arcsec)', fontsize = 3 ,labelpad = 1) 
    plt.ylabel('Ellipticity', fontsize = 3, labelpad = 1)
    plt.xticks()
    plt.yticks()
    plt.tick_params(axis = 'x', direction = 'out', length = 1.5, width = 0.5, labelsize = 3, pad = 1)
    plt.tick_params(axis = 'y', direction = 'out', length = 1.5, width = 0.5, labelsize = 3, pad = 1)
    plt.legend(loc = 'best', fontsize = 3.5)

    plt.subplot(5,3,k)
    plt.errorbar(globals()['df_my_eps_{}'.format(f)]['R"'][10:], globals()['df_my_eps_{}'.format(f)]['PA'][10:], globals()['df_my_eps_{}'.format(f)]['PA_error'][10:], fmt='r^-', ms=0.5, linewidth=0.2, elinewidth=0.2, ecolor='black', capsize=1, capthick=0.5, label= f)
    plt.xlabel('Major_R(arcsec)', fontsize = 3 ,labelpad = 1)
    plt.ylabel('Position Angle (deg)', fontsize = 3, labelpad = 1)
    plt.xticks()
    plt.yticks()
    plt.tick_params(axis = 'x', direction = 'out', length = 1.5, width = 0.5, labelsize = 3, pad = 1)
    plt.tick_params(axis = 'y', direction = 'out', length = 1.5, width = 0.5, labelsize = 3, pad = 1)
    plt.legend(loc = 'best', fontsize = 3.5)

plt.show()

In [ ]:
# 이미지 저장

save_file_name = str(ring_num)+'_4_result_graph(with error).jpg'
plt.savefig(save_path + save_file_name)

* 에러 값을 미포함한 광도 윤곽의 광도와 타원율, 위치각의 변화를 분석함으로써 은하의 특징을 살펴봅니다.

In [ ]:
# 그래프 시각화 에러 값 미포함

plt.ion()
plt.figure(figsize=(6, 3.5), dpi=300)
for i,j,k,f in zip(range(1,17,3), range(2,18,3), range(3,19,3), filter):
    plt.subplot(5,3,i)
    plt.plot(globals()['df_my_eps_{}'.format(f)]['R"'][10:], globals()['df_my_eps_{}'.format(f)]['Mu'][10:], 'r^-', ms=0.5, linewidth=0.2, label= f)
    plt.xlabel('Major_R(arcsec)', fontsize = 3 ,labelpad = 1) 
    plt.ylabel('Magnitude', fontsize = 3, labelpad = 1)
    plt.xticks()
    plt.yticks()
    plt.tick_params(axis = 'x', direction = 'out', length = 1.5, width = 0.5, labelsize = 3, pad = 1)
    plt.tick_params(axis = 'y', direction = 'out', length = 1.5, width = 0.5, labelsize = 3, pad = 1)
    plt.gca().invert_yaxis()
    plt.legend(loc = 'best', fontsize = 3.5)

    plt.subplot(5,3,j)
    plt.plot(globals()['df_my_eps_{}'.format(f)]['R"'][10:], globals()['df_my_eps_{}'.format(f)]['Ellipticity'][10:], 'r^-', ms=0.5, linewidth=0.2, label= f)
    plt.xlabel('Major_R(arcsec)', fontsize = 3 ,labelpad = 1) 
    plt.ylabel('Ellipticity', fontsize = 3, labelpad = 1)
    plt.xticks()
    plt.yticks()
    plt.tick_params(axis = 'x', direction = 'out', length = 1.5, width = 0.5, labelsize = 3, pad = 1)
    plt.tick_params(axis = 'y', direction = 'out', length = 1.5, width = 0.5, labelsize = 3, pad = 1)
    plt.legend(loc = 'best', fontsize = 3.5)

    plt.subplot(5,3,k)
    plt.plot(globals()['df_my_eps_{}'.format(f)]['R"'][10:], globals()['df_my_eps_{}'.format(f)]['PA'][10:], 'r^-', ms=0.5, linewidth=0.2, label= f)
    plt.xlabel('Major_R(arcsec)', fontsize = 3 ,labelpad = 1)
    plt.ylabel('Position Angle (deg)', fontsize = 3, labelpad = 1)
    plt.xticks()
    plt.yticks()
    plt.tick_params(axis = 'x', direction = 'out', length = 1.5, width = 0.5, labelsize = 3, pad = 1)
    plt.tick_params(axis = 'y', direction = 'out', length = 1.5, width = 0.5, labelsize = 3, pad = 1)
    plt.legend(loc = 'best', fontsize = 3.5)

plt.show()

In [ ]:
# 이미지 저장

save_file_name = str(ring_num)+'_5_result_graph(without error).jpg'
plt.savefig(save_path + save_file_name)

### **랜덤포레스트 회귀분석을 통한 등광도 특징 보조 분석**

* 랜덤포레스트 회귀분석을 통한 등광도 특징 보조 분석은 실제 등광도선과 분석된 등광도선의 불일치가 심할때 합니다.
* 이 분석은 등광도 맞춤 분석의 옵션 조정 후 이상값 회피를 통한 회귀분석으로, 더 정확한 등광도 관련 광도 윤곽, 타원율, 위치각을 도출해 냅니다.  

In [ ]:
# 머신러닝으로 분석할 필터 선택, 변수 설정

sel = str(input("분석하려는 필터를 입력하세요.")) # 필터 선택
r = globals()['df_my_eps_{}'.format(sel)][['R"']] # 거리 변수 설정
mu = globals()['df_my_eps_{}'.format(sel)].Mu # 광도 변수 설정
ep = globals()['df_my_eps_{}'.format(sel)].Ellipticity # 타원율 변수 설정
pa = globals()['df_my_eps_{}'.format(sel)].PA # 위치각 변수 설정

In [ ]:
# 머신러닝 랜덤포레스트 회귀

# 랜덤포레스트 초매개변수 설정
rfr = RandomForestRegressor()

param_grid = {'n_estimators': [1000], 'max_depth' : [4], 'min_samples_leaf' : [5], 'bootstrap': [True]}  # 초매개변수 설정
grid_search = GridSearchCV(rfr, param_grid, scoring='neg_mean_squared_error', return_train_score=True) # 성능 측정 지표 설정

# 랜덤포레스트 훈련 및 예측
grid_search.fit(r, mu) # 밝기 강도 예측
y0_pred = grid_search.predict(r)

grid_search.fit(r, ep) # 타원율 예측
y1_pred = grid_search.predict(r)

grid_search.fit(r, pa) # 위치각 예측
y2_pred = grid_search.predict(r)

In [ ]:
# 머신러닝 반영 회귀 그래프

plt.ion()
plt.figure(figsize=(4,4.5), dpi=300)
for f in sel:
    plt.subplot(3,1,1)
    plt.plot(globals()['df_my_eps_{}'.format(f)]['R"'][10:], globals()['df_my_eps_{}'.format(f)]['Mu'][10:], 'r^-', ms=7, mfc='none', linewidth=1, alpha = 0.5, label= f + '_Ori')
    plt.plot(r[10:], y0_pred[10:], 'gx:', ms=7, mec='green', mew=1, mfc='none', linewidth=1.5, alpha = 1, label= f + '_RFR')
    plt.ylabel('Magnitude', fontsize = 9.5, labelpad = 1)
    plt.xticks()
    plt.yticks()
    plt.tick_params(axis = 'x', direction = 'out', length = 3.5, width = 2, labelsize = 9.5, pad = 1)
    plt.tick_params(axis = 'y', direction = 'out', length = 3.5, width = 2, labelsize = 9.5, pad = 1)
    plt.gca().invert_yaxis()
    plt.legend(loc = 'best', fontsize = 9.5)

    plt.subplot(3,1,2)
    #plt.errorbar(globals()['df_my_eps_{}'.format(f)]['R"'][10:], globals()['df_my_eps_{}'.format(f)]['Ellipticity'][10:], globals()['df_my_eps_{}'.format(f)]['Ellip_error'][10:], fmt='r^-', ms=7, mfc='none', linewidth=1, elinewidth=1, ecolor='black', capsize=2, capthick=2.5, alpha = 0.5, label= f + '_Ori')
    plt.plot(globals()['df_my_eps_{}'.format(f)]['R"'][10:], globals()['df_my_eps_{}'.format(f)]['Ellipticity'][10:], 'r^-', ms=7, mfc='none', linewidth=1, alpha = 0.5, label= f + '_Ori')
    plt.plot(r[10:], y1_pred[10:], 'gx:', ms=7, mec='green', mew=1, mfc='none', linewidth=1.5, alpha = 1, label= f + '_RFR')

    plt.ylabel('Ellipticity', fontsize = 9.5, labelpad = 1)
    plt.xticks()
    plt.yticks()
    plt.tick_params(axis = 'x', direction = 'out', length = 3.5, width = 2, labelsize = 9.5, pad = 1)
    plt.tick_params(axis = 'y', direction = 'out', length = 3.5, width = 2, labelsize = 9.5, pad = 1)
    plt.legend(loc = 'upper right', fontsize = 9.5)

    plt.subplot(3,1,3)
    #plt.errorbar(globals()['df_my_eps_{}'.format(f)]['R"'][10:], globals()['df_my_eps_{}'.format(f)]['PA'][10:], globals()['df_my_eps_{}'.format(f)]['PA_error'][10:], fmt='r^-', ms=7, mfc='none', linewidth=1, elinewidth=1, ecolor='black', capsize=2, capthick=2.5, alpha = 0.5, label= f + '_Ori')
    plt.plot(globals()['df_my_eps_{}'.format(f)]['R"'][10:], globals()['df_my_eps_{}'.format(f)]['PA'][10:], 'r^-', ms=7, mfc='none', linewidth=1, alpha = 0.5, label= f + '_Ori')
    plt.plot(r[10:], y2_pred[10:], 'gx:', ms=7, mec='green', mew=1, mfc='none', linewidth=1.5, alpha = 1, label= f + '_RFR')
    plt.xlabel('Major_R(arcsec)', fontsize = 9.5,labelpad = 1)
    plt.ylabel('Position Angle (deg)', fontsize = 9.5, labelpad = 1)
    plt.xticks()
    plt.yticks()
    plt.tick_params(axis = 'x', direction = 'out', length = 3.5, width = 2, labelsize = 9.5, pad = 1)
    plt.tick_params(axis = 'y', direction = 'out', length = 3.5, width = 2, labelsize = 9.5, pad = 1)
    plt.legend(loc = 'right', fontsize = 9.5)

    plt.show()

In [ ]:
# 이미지 저장

save_file_name = str(ring_num)+'_6_regression_result_graph.jpg'
plt.savefig(save_path + save_file_name)